# Notebook 2: 特征工程 / Feature Engineering

本笔记本教你如何将原始价格数据转换为机器学习可用的特征
This notebook teaches you how to transform raw price data into features usable by machine learning

## 学习目标 / Learning Objectives
- 理解特征工程的概念 / Understand the concept of feature engineering
- 学习计算技术指标 / Learn to calculate technical indicators
- 创建预测标签 / Create prediction labels
- 准备机器学习训练数据 / Prepare ML training data

In [ ]:
# 导入必要的库 / Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# 添加项目路径 / Add project path
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

from utils.binance_client import BinanceUtility
from utils.data_processor import DataProcessor
from utils.visualizer import DataVisualizer

print("✅ 库导入成功 / Libraries imported successfully")

## 步骤 1: 加载数据 / Step 1: Load Data

In [ ]:
# 获取数据 / Fetch data
client = BinanceUtility()
symbol = 'BTCUSDT'

print(f"📡 正在获取 {symbol} 数据... / Fetching {symbol} data...")
df = client.fetch_historical_data(symbol, '1h', '3 months ago UTC')

print(f"✅ 数据加载成功！/ Data loaded successfully!")
print(f"📊 数据量 / Data size: {len(df)} rows")

## 步骤 2: 理解特征工程 / Step 2: Understand Feature Engineering

In [ ]:
print("""📖 什么是特征工程？/ What is Feature Engineering?

特征工程是将原始数据转换为机器学习模型更容易理解的特征的过程。
Feature engineering is the process of transforming raw data into features that ML models can understand better.

在币价预测中，我们不能直接使用价格数值，因为：
In price prediction, we cannot directly use price values because:

1. 价格数值很大且不稳定 / Price values are large and unstable
2. 价格本身不包含趋势信息 / Price alone doesn't contain trend information
3. 需要从价格中提取有意义的模式 / Need to extract meaningful patterns from prices

🎯 我们使用的特征包括：/ Features we use include:
   • SMA (简单移动平均线) / Simple Moving Average
   • RSI (相对强弱指数) / Relative Strength Index
   • ROC (变化率) / Rate of Change
   • Volatility (波动率) / Volatility
""")

## 步骤 3: 添加技术指标 / Step 3: Add Technical Indicators

In [ ]:
# 创建数据处理器 / Create data processor
processor = DataProcessor()

print("🔧 正在进行特征工程... / Performing feature engineering...")
df_features = processor.add_technical_indicators(df)

print(f"✅ 特征工程完成！/ Feature engineering complete!")
print(f"📊 原始特征 / Original features: {df.shape[1]}")
print(f"📊 新增特征 / New features: {df_features.shape[1]}")

## 步骤 4: 查看新增特征 / Step 4: Examine New Features

In [ ]:
# 显示包含特征的最新数据 / Display latest data with features
print("📋 数据预览（包含特征）/ Data Preview (with features):")
df_features[['timestamp', 'close', 'sma_7', 'sma_25', 'rsi_14', 'roc', 'volatility']].tail()

## 步骤 5: 理解每个特征 / Step 5: Understand Each Feature

In [ ]:
print("""📚 特征详解 / Feature Explanation:

1️⃣ SMA (Simple Moving Average - 简单移动平均线)
   • 计算：过去 N 个周期的平均价格 / Calculation: Average price over past N periods
   • 作用：平滑价格波动，显示趋势 / Purpose: Smooth price fluctuations, show trend
   • SMA_7: 短期趋势（7小时平均）/ Short-term trend (7-hour average)
   • SMA_25: 长期趋势（25小时平均）/ Long-term trend (25-hour average)

2️⃣ RSI (Relative Strength Index - 相对强弱指数)
   • 范围：0-100 / Range: 0-100
   • >70: 超买（可能下跌）/ Overbought (may go down)
   • <30: 超卖（可能上涨）/ Oversold (may go up)
   • 作用：判断是否过度买入或卖出 / Purpose: Determine if overbought or oversold

3️⃣ ROC (Rate of Change - 变化率)
   • 计算：(当前价格 - N个周期前的价格) / N个周期前的价格
   • 作用：衡量价格变化速度 / Purpose: Measure price change speed
   • 正值：价格上涨 / Positive: Price going up
   • 负值：价格下跌 / Negative: Price going down

4️⃣ Volatility (波动率)
   • 计算：过去 N 个周期的价格标准差 / Calculation: Standard deviation of prices over N periods
   • 作用：衡量价格波动程度 / Purpose: Measure price fluctuation level
   • 高值：价格波动大 / High: Large price fluctuations
   • 低值：价格稳定 / Low: Price is stable
""")

## 步骤 6: 可视化特征 / Step 6: Visualize Features

In [ ]:
# 使用可视化工具 / Use visualizer
visualizer = DataVisualizer()

# 绘制价格和移动平均线 / Plot price and moving averages
visualizer.plot_technical_indicators(
    df_features.tail(168),  # 最近 7 天的数据 / Last 7 days data
    indicators=['sma_7', 'sma_25']
)

In [ ]:
# 绘制 RSI 指标 / Plot RSI indicator
visualizer.plot_rsi(df_features.tail(168))

## 步骤 7: 特征统计分析 / Step 7: Feature Statistics Analysis

In [ ]:
# 计算特征统计信息 / Calculate feature statistics
feature_cols = ['sma_7', 'sma_25', 'rsi_14', 'roc', 'volatility']
feature_stats = df_features[feature_cols].describe()

print("📊 特征统计 / Feature Statistics:")
print(feature_stats.round(2))

In [ ]:
# 查看特征相关性 / Check feature correlation
print("📊 特征相关性矩阵 / Feature Correlation Matrix:")
correlation_matrix = df_features[feature_cols].corr()
print(correlation_matrix.round(3))

## 步骤 8: 创建预测标签 / Step 8: Create Prediction Labels

In [ ]:
print("""📖 预测标签说明 / Prediction Label Explanation:

机器学习需要"标签"（目标变量）来学习。
Machine learning needs "labels" (target variables) to learn.

在这个项目中，我们预测的是：
In this project, we predict:

🎯 "下个周期的价格是上涨还是下跌？"
   "Will the price go up or down in the next period?"

标签定义 / Label Definition:
   • 1 = 上涨 (UP): 下个周期的收盘价 > 当前收盘价
   • 0 = 下跌 (DOWN): 下个周期的收盘价 <= 当前收盘价
""")

In [ ]:
# 准备特征和标签 / Prepare features and labels
X, y = processor.prepare_features_labels(df_features)

print(f"✅ 特征和标签准备完成！/ Features and labels prepared!")
print(f"📊 特征矩阵形状 / Feature matrix shape: {X.shape}")
print(f"📊 标签向量形状 / Label vector shape: {y.shape}")

## 步骤 9: 分析标签分布 / Step 9: Analyze Label Distribution

In [ ]:
# 统计标签分布 / Count label distribution
label_counts = pd.Series(y).value_counts()
label_percentages = pd.Series(y).value_counts(normalize=True) * 100

print("📊 标签分布 / Label Distribution:")
print(f"   下跌 (DOWN/0): {label_counts[0]} 次 ({label_percentages[0]:.2f}%)")
print(f"   上涨 (UP/1): {label_counts[1]} 次 ({label_percentages[1]:.2f}%)")

# 可视化标签分布 / Visualize label distribution
plt.figure(figsize=(8, 6))
plt.bar(['DOWN (0)', 'UP (1)'], label_counts.values, color=['red', 'green'], alpha=0.7)
plt.title('Label Distribution / 标签分布', fontsize=14, fontweight='bold')
plt.xlabel('Label / 标签', fontsize=12)
plt.ylabel('Count / 数量', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 提示 / Tip: 如果标签分布不均衡（一个类远多于另一个），可能需要使用特殊技术处理。")

## 步骤 10: 查看特征列表 / Step 10: View Feature List

In [ ]:
print("📋 完整特征列表 / Complete Feature List:")
print("=" * 50)
for i, col in enumerate(X.columns, 1):
    print(f"{i:2d}. {col}")
print("=" * 50)
print(f"总共 / Total: {len(X.columns)} 个特征 / features")

## 步骤 11: 保存处理后的数据 / Step 11: Save Processed Data

In [ ]:
# 保存处理后的数据 / Save processed data
os.makedirs('data', exist_ok=True)
output_path = f'data/{symbol}_features.csv'
df_features.to_csv(output_path, index=False)

print(f"✅ 处理后的数据已保存到 / Processed data saved to: {output_path}")
print(f"📁 文件大小 / File size: {os.path.getsize(output_path) / 1024:.2f} KB")

## 🎓 总结 / Summary

在本笔记本中，你学会了：
In this notebook, you learned:

1. ✅ 理解特征工程的重要性
2. ✅ 计算移动平均线 (SMA)
3. ✅ 计算相对强弱指数 (RSI)
4. ✅ 计算变化率和波动率
5. ✅ 可视化技术指标
6. ✅ 创建预测标签（上涨/下跌）
7. ✅ 分析特征统计和分布

💡 关键要点 / Key Takeaways:
- 特征工程是机器学习的核心步骤
- 技术指标帮助模型理解价格模式
- 标签定义了我们要预测的目标

下一步：训练机器学习模型 / Next Step: Train ML Model (Notebook 3)